# 📊 Comparador de Documentos PDF con LangChain y vLLM
 
Este notebook permite comparar dos archivos PDF usando:
- Análisis básico con difflib
- Análisis semántico con sentence-transformers  
- **Análisis inteligente con LangChain + vLLM/OpenAI/Ollama**

## 📦 Instalación de dependencias:
```bash
pip install PyPDF2 sentence-transformers scikit-learn nltk pandas numpy matplotlib seaborn
pip install langchain langchain-community langchain-openai
pip install vllm  # Para servidor vLLM local
```

## 🚀 BLOQUE 1: Importaciones y Configuración Inicial

In [ ]:
import PyPDF2
import difflib
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import pandas as pd
import time
import warnings
from pathlib import Path
from IPython.display import display, Markdown, HTML
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os


# LangChain imports
from langchain.llms import VLLMOpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import HumanMessage, SystemMessage
from langchain_community.llms import Ollama
from langchain.callbacks import StdOutCallbackHandler

# Configurar warnings
warnings.filterwarnings('ignore')

print("✅ BLOQUE 1 - Importaciones completadas")

## 📄 BLOQUE 2: Configuración de Paths y Modelos

In [ ]:
# 🔧 CONFIGURA AQUÍ LAS RUTAS DE TUS PDFS
PDF1_PATH = "Red_Hat_OpenShift_AI_Self-Managed-2.19-Release_notes-en-US.pdf"
PDF2_PATH = "Red_Hat_OpenShift_AI_Self-Managed-2.21-Release_notes-en-US.pdf"

# Ejemplo de rutas (modifica según tu caso):
# PDF1_PATH = "documentos/contrato_original.pdf"
# PDF2_PATH = "documentos/contrato_modificado.pdf"

print(f"📄 PDF 1: {PDF1_PATH}")
print(f"📄 PDF 2: {PDF2_PATH}")

# CONFIGURACIÓN DE MODELOS LLM CON LANGCHAIN
# Opciones de configuración de LLM
LLM_PROVIDER = "vllm"  # Opciones: "vllm", "openai", "ollama"

# Configuración vLLM
VLLM_CONFIG = {
    "openai_api_base": "http://granite-31-8b-instruct-predictor.comparador-docs.svc.cluster.local:8080/v1",
    "openai_api_key": "EMPTY", 
    "model_name": "granite-31-8b-instruct",
    "temperature": 0.3,
    "max_tokens": 2048
}

# Configuración OpenAI
# OPENAI_CONFIG = {
#     "api_key": "tu_openai_api_key_aqui",  # Reemplaza con tu API key
#     "model": "gpt-3.5-turbo",
#     "temperature": 0.3,
#     "max_tokens": 2048
# }

# Configuración Ollama
# OLLAMA_CONFIG = {
#     "model": "llama2",  # O cualquier modelo que tengas en Ollama
#     "temperature": 0.3,
#     "base_url": "http://localhost:11434"  # URL por defecto de Ollama
# }

print(f"🤖 Proveedor LLM configurado: {LLM_PROVIDER}")

def verificar_archivos(path1, path2):
    """Verifica que ambos archivos PDF existan"""
    errores = []
    
    if not Path(path1).exists():
        errores.append(f"❌ No se encuentra: {path1}")
    
    if not Path(path2).exists():
        errores.append(f"❌ No se encuentra: {path2}")
    
    if errores:
        for error in errores:
            print(error)
        return False
    else:
        print("✅ Ambos archivos encontrados")
        return True

# Verificar archivos
archivos_ok = verificar_archivos(PDF1_PATH, PDF2_PATH)
print("✅ BLOQUE 2 - Configuración completada")

## 🤖 BLOQUE 3: Configuración de LangChain LLM

In [ ]:
def inicializar_llm(provider="vllm"):
    """
    Inicializa el modelo LLM usando LangChain según el proveedor
    
    Args:
        provider: "vllm", "openai", o "ollama"
        
    Returns:
        Objeto LLM de LangChain
    """
    try:
        if provider == "vllm":
            print("🚀 Inicializando vLLM con LangChain...")
            
            # Verificar conexión con vLLM
            import requests
            try:
                response = requests.get(f"{VLLM_CONFIG['openai_api_base'].replace('/v1', '')}/health", timeout=20)
                if response.status_code != 200:
                    raise Exception("vLLM no responde")
                print("✅ vLLM servidor conectado")
            except:
                print("❌ Error conectando con vLLM. Asegúrate de que esté ejecutándose:")
                print(f"   python -m vllm.entrypoints.openai.api_server --model {VLLM_CONFIG['model_name']} --port 8000")
                return None
            
            llm = VLLMOpenAI(
                openai_api_base=VLLM_CONFIG["openai_api_base"],
                openai_api_key=VLLM_CONFIG["openai_api_key"],
                model_name=VLLM_CONFIG["model_name"],
                temperature=VLLM_CONFIG["temperature"],
                max_tokens=VLLM_CONFIG["max_tokens"]
            )
            
        elif provider == "openai":
            print("🚀 Inicializando OpenAI con LangChain...")
            
            if OPENAI_CONFIG["api_key"] == "tu_openai_api_key_aqui":
                print("❌ Por favor configura tu API key de OpenAI en OPENAI_CONFIG")
                return None
            
            llm = ChatOpenAI(
                api_key=OPENAI_CONFIG["api_key"],
                model=OPENAI_CONFIG["model"],
                temperature=OPENAI_CONFIG["temperature"],
                max_tokens=OPENAI_CONFIG["max_tokens"]
            )
            
        elif provider == "ollama":
            print("🚀 Inicializando Ollama con LangChain...")
            
            # Verificar conexión con Ollama
            import requests
            try:
                response = requests.get(f"{OLLAMA_CONFIG['base_url']}/api/tags", timeout=5)
                if response.status_code != 200:
                    raise Exception("Ollama no responde")
                print("✅ Ollama servidor conectado")
            except:
                print("❌ Error conectando con Ollama. Asegúrate de que esté ejecutándose:")
                print("   ollama serve")
                return None
            
            llm = Ollama(
                model=OLLAMA_CONFIG["model"],
                temperature=OLLAMA_CONFIG["temperature"],
                base_url=OLLAMA_CONFIG["base_url"]
            )
            
        else:
            print(f"❌ Proveedor no soportado: {provider}")
            return None
        
        print(f"✅ LLM inicializado exitosamente: {provider}")
        return llm
        
    except Exception as e:
        print(f"❌ Error inicializando LLM {provider}: {str(e)}")
        return None

# Inicializar el LLM
llm_model = inicializar_llm(LLM_PROVIDER)
llm_disponible = llm_model is not None
print("✅ BLOQUE 3 - LangChain LLM configurado")

## 📝 BLOQUE 4: Templates de Prompts con LangChain

In [ ]:
def crear_templates_langchain():
    """Crea templates de prompts optimizados para LangChain"""
    
    # Template para análisis comprehensivo
    template_comprehensive = PromptTemplate(
        input_variables=["documento1", "documento2"],
        template="""Eres un experto analista de documentos. Analiza y compara los siguientes dos documentos PDF de manera detallada y estructurada.

DOCUMENTO 1:
{documento1}

DOCUMENTO 2:
{documento2}

Proporciona tu análisis en el siguiente formato estructurado:

**SIMILITUDES PRINCIPALES:**
- [Lista 3-5 similitudes clave entre los documentos]

**DIFERENCIAS PRINCIPALES:**
- [Lista 3-5 diferencias importantes entre los documentos]

**TEMAS COMUNES:**
- [Identifica temas que aparecen en ambos documentos]

**TIPO DE DOCUMENTOS:**
- [Describe qué tipo de documentos son y su propósito]

**ANÁLISIS DE ESTRUCTURA:**
- [Compara la organización y estructura de ambos documentos]

**PUNTUACIÓN DE SIMILITUD:** [Número del 0 al 100]

**RECOMENDACIONES:**
- [Sugerencias basadas en la comparación]

**RESUMEN EJECUTIVO:**
[Resumen conciso de 2-3 líneas sobre la comparación]"""
    )
    
    # Template para análisis rápido
    template_quick = PromptTemplate(
        input_variables=["documento1", "documento2"],
        template="""Compara estos dos documentos y proporciona un análisis rápido y conciso:

DOCUMENTO 1: {documento1}

DOCUMENTO 2: {documento2}

Responde en el siguiente formato:
1. Similitud general (0-100): [número]
2. Tipo de documentos: [descripción breve]
3. Similitudes principales (máximo 3):
   - [similitud 1]
   - [similitud 2]
   - [similitud 3]
4. Diferencias principales (máximo 3):
   - [diferencia 1]
   - [diferencia 2]
   - [diferencia 3]
5. Recomendación: [una línea de recomendación]"""
    )
    
    # Template para análisis específico por dominio
    template_domain = PromptTemplate(
        input_variables=["documento1", "documento2", "dominio"],
        template="""Eres un experto en {dominio}. Analiza estos documentos desde la perspectiva de {dominio}:

DOCUMENTO 1:
{documento1}

DOCUMENTO 2:
{documento2}

Proporciona un análisis especializado que incluya:
1. Elementos específicos de {dominio} presentes en cada documento
2. Cumplimiento de estándares o mejores prácticas de {dominio}
3. Diferencias técnicas relevantes para {dominio}
4. Recomendaciones específicas para {dominio}
5. Puntuación de similitud considerando aspectos de {dominio} (0-100)"""
    )
    
    return {
        "comprehensive": template_comprehensive,
        "quick": template_quick,
        "domain": template_domain
    }

# Crear templates
prompt_templates = crear_templates_langchain()
print("✅ BLOQUE 4 - Templates de prompts creados")

## 🛠️ BLOQUE 5: Configuración de NLTK y Modelos

In [ ]:
def configurar_nltk():
    """Configura y descarga recursos necesarios de NLTK"""
    try:
        nltk.data.find('tokenizers/punkt')
        print("✅ NLTK punkt ya disponible")
    except LookupError:
        print("📦 Descargando NLTK punkt...")
        nltk.download('punkt', quiet=True)
    
    try:
        nltk.data.find('tokenizers/punkt_tab')
        print("✅ NLTK punkt_tab ya disponible")
    except LookupError:
        print("📦 Descargando NLTK punkt_tab...")
        nltk.download('punkt_tab', quiet=True)
    
    try:
        nltk.data.find('corpora/stopwords')
        print("✅ NLTK stopwords ya disponible")
    except LookupError:
        print("📦 Descargando NLTK stopwords...")
        nltk.download('stopwords', quiet=True)
    
    return set(stopwords.words('spanish'))

def cargar_modelo_embeddings():
    """Carga el modelo de embeddings semánticos"""
    print("🔄 Cargando modelo de embeddings...")
    modelo = SentenceTransformer('all-MiniLM-L6-v2')
    print("✅ Modelo de embeddings cargado")
    return modelo

# Configurar recursos
stop_words = configurar_nltk()
modelo_embeddings = cargar_modelo_embeddings()
print("✅ BLOQUE 5 - NLTK y modelos configurados")

## 📚 BLOQUE 6: Funciones de Extracción de PDF

In [ ]:
def extraer_texto_pdf(ruta_pdf):
    """
    Extrae texto de un archivo PDF
    
    Args:
        ruta_pdf: Ruta al archivo PDF
        
    Returns:
        dict: Información extraída del PDF
    """
    try:
        ruta = Path(ruta_pdf)
        
        if not ruta.exists():
            return {
                "error": f"El archivo {ruta} no existe",
                "texto": "",
                "paginas": 0,
                "exito": False
            }
        
        if ruta.suffix.lower() != '.pdf':
            return {
                "error": f"El archivo no es un PDF",
                "texto": "",
                "paginas": 0,
                "exito": False
            }
        
        texto_completo = ""
        with open(ruta, 'rb') as archivo:
            lector_pdf = PyPDF2.PdfReader(archivo)
            num_paginas = len(lector_pdf.pages)
            
            for num_pag, pagina in enumerate(lector_pdf.pages):
                try:
                    texto_pagina = pagina.extract_text()
                    if texto_pagina:
                        texto_completo += f"\n--- Página {num_pag + 1} ---\n"
                        texto_completo += texto_pagina
                except Exception as e:
                    print(f"⚠️ Error extrayendo página {num_pag + 1}: {e}")
        
        return {
            "texto": texto_completo.strip(),
            "paginas": num_paginas,
            "tamaño_archivo": ruta.stat().st_size,
            "nombre_archivo": ruta.name,
            "exito": True
        }
        
    except Exception as e:
        return {
            "error": f"Error procesando PDF: {str(e)}",
            "texto": "",
            "paginas": 0,
            "exito": False
        }

def extraer_metadatos_pdf(ruta_pdf):
    """Extrae metadatos del PDF"""
    try:
        with open(ruta_pdf, 'rb') as archivo:
            lector_pdf = PyPDF2.PdfReader(archivo)
            metadatos = lector_pdf.metadata
            
            return {
                "titulo": metadatos.get('/Title', 'N/A') if metadatos else 'N/A',
                "autor": metadatos.get('/Author', 'N/A') if metadatos else 'N/A',
                "creador": metadatos.get('/Creator', 'N/A') if metadatos else 'N/A',
                "fecha_creacion": str(metadatos.get('/CreationDate', 'N/A')) if metadatos else 'N/A',
                "paginas": len(lector_pdf.pages)
            }
    except Exception as e:
        return {"error": f"Error extrayendo metadatos: {str(e)}"}

print("✅ BLOQUE 6 - Funciones de extracción PDF creadas")

## 🧹 BLOQUE 7: Funciones de Preprocesamiento

In [ ]:
def limpiar_texto(texto):
    """
    Limpia y preprocesa el texto extraído del PDF
    
    Args:
        texto: Texto a limpiar
        
    Returns:
        str: Texto limpio
    """
    if not texto:
        return ""
    
    # Remover marcadores de página
    texto = re.sub(r'--- Página \d+ ---', '', texto)
    
    # Limpiar espacios múltiples
    texto = re.sub(r'\s+', ' ', texto)
    
    # Limpiar saltos de línea múltiples
    texto = re.sub(r'\n+', '\n', texto)
    
    return texto.strip()

def obtener_estadisticas_texto(texto):
    """Obtiene estadísticas básicas del texto"""
    if not texto:
        return {
            "caracteres": 0,
            "palabras": 0,
            "oraciones": 0,
            "palabras_unicas": 0
        }
    
    palabras = word_tokenize(texto)
    oraciones = sent_tokenize(texto)
    palabras_unicas = set(palabras)
    
    return {
        "caracteres": len(texto),
        "palabras": len(palabras),
        "oraciones": len(oraciones),
        "palabras_unicas": len(palabras_unicas)
    }

def truncar_texto_para_llm(texto, max_chars=4000):
    """Trunca texto para no exceder límites del LLM"""
    if len(texto) <= max_chars:
        return texto
    
    # Truncar y agregar indicador
    return texto[:max_chars] + "\n\n[TEXTO TRUNCADO...]"

print("✅ BLOQUE 7 - Funciones de preprocesamiento creadas")

## 📊 BLOQUE 8: Análisis Básico y Semántico

In [ ]:
def analisis_basico_difflib(texto1, texto2):
    """
    Realiza análisis básico de diferencias usando difflib
    
    Args:
        texto1, texto2: Textos a comparar
        
    Returns:
        dict: Resultados del análisis básico
    """
    if not texto1 or not texto2:
        return {
            "error": "Uno o ambos textos están vacíos",
            "ratio_similitud": 0.0
        }
    
    # Dividir en líneas
    lineas1 = texto1.splitlines()
    lineas2 = texto2.splitlines()
    
    # Calcular diferencias
    diferencias = list(difflib.unified_diff(
        lineas1, lineas2,
        fromfile='PDF 1',
        tofile='PDF 2',
        lineterm=''
    ))
    
    # Ratio de similitud
    ratio_similitud = difflib.SequenceMatcher(None, texto1, texto2).ratio()
    
    # Contar cambios
    adiciones = len([linea for linea in diferencias if linea.startswith('+')])
    eliminaciones = len([linea for linea in diferencias if linea.startswith('-')])
    
    return {
        "ratio_similitud": ratio_similitud,
        "total_lineas_diff": len(diferencias),
        "adiciones": adiciones,
        "eliminaciones": eliminaciones,
        "resumen_cambios": f"{adiciones} adiciones, {eliminaciones} eliminaciones",
        "porcentaje_similitud": ratio_similitud * 100
    }

def analisis_semantico(texto1, texto2, modelo_embeddings):
    """
    Realiza análisis de similitud semántica usando embeddings
    
    Args:
        texto1, texto2: Textos a comparar
        modelo_embeddings: Modelo SentenceTransformer cargado
        
    Returns:
        dict: Resultados del análisis semántico
    """
    if not texto1 or not texto2:
        return {
            "error": "Uno o ambos textos están vacíos",
            "similitud_documento": 0.0
        }
    
    try:
        # Embeddings de documentos completos
        embeddings = modelo_embeddings.encode([texto1, texto2])
        similitud_documento = cosine_similarity([embeddings[0]], [embeddings[1]])[0][0]
        
        # Análisis por oraciones (limitado para evitar sobrecarga de memoria)
        oraciones1 = sent_tokenize(texto1)[:50]  # Máximo 50 oraciones
        oraciones2 = sent_tokenize(texto2)[:50]
        
        if oraciones1 and oraciones2:
            embeddings_oraciones1 = modelo_embeddings.encode(oraciones1)
            embeddings_oraciones2 = modelo_embeddings.encode(oraciones2)
            
            matriz_similitud = cosine_similarity(embeddings_oraciones1, embeddings_oraciones2)
            similitud_promedio_oraciones = np.mean(matriz_similitud)
            similitud_maxima_oraciones = np.max(matriz_similitud)
        else:
            similitud_promedio_oraciones = 0.0
            similitud_maxima_oraciones = 0.0
        
        return {
            "similitud_documento": float(similitud_documento),
            "similitud_promedio_oraciones": float(similitud_promedio_oraciones),
            "similitud_maxima_oraciones": float(similitud_maxima_oraciones),
            "oraciones_analizadas": min(len(oraciones1), len(oraciones2)),
            "porcentaje_similitud": float(similitud_documento) * 100
        }
        
    except Exception as e:
        return {
            "error": f"Error en análisis semántico: {str(e)}",
            "similitud_documento": 0.0
        }

print("✅ BLOQUE 8 - Funciones de análisis básico y semántico creadas")

## 🧠 BLOQUE 9: Análisis con LangChain

In [ ]:
def analisis_langchain(texto1, texto2, tipo_analisis="comprehensive", dominio=None):
    """
    Realiza análisis usando LangChain con el LLM configurado
    
    Args:
        texto1, texto2: Textos a comparar
        tipo_analisis: "comprehensive", "quick", o "domain"
        dominio: Dominio específico para análisis especializado
        
    Returns:
        dict: Resultados del análisis LangChain
    """
    if not llm_disponible:
        return {"error": "LLM no está disponible"}
    
    # Truncar textos para no exceder límites
    texto1_truncado = truncar_texto_para_llm(texto1)
    texto2_truncado = truncar_texto_para_llm(texto2)
    
    print(f"🤖 Ejecutando análisis LangChain: {tipo_analisis}")
    tiempo_inicio = time.time()
    
    try:
        # Seleccionar template y crear chain
        if tipo_analisis == "domain" and dominio:
            template = prompt_templates["domain"]
            chain = LLMChain(
                llm=llm_model, 
                prompt=template,
                callbacks=[StdOutCallbackHandler()] if LLM_PROVIDER == "vllm" else []
            )
            inputs = {
                "documento1": texto1_truncado,
                "documento2": texto2_truncado,
                "dominio": dominio
            }
        else:
            template_key = "comprehensive" if tipo_analisis == "comprehensive" else "quick"
            template = prompt_templates[template_key]
            chain = LLMChain(
                llm=llm_model, 
                prompt=template,
                callbacks=[StdOutCallbackHandler()] if LLM_PROVIDER == "vllm" else []
            )
            inputs = {
                "documento1": texto1_truncado,
                "documento2": texto2_truncado
            }
        
        # Ejecutar chain
        resultado = chain.run(inputs)
        
        tiempo_procesamiento = time.time() - tiempo_inicio
        
        # Parsear resultado
        analisis_parseado = parsear_respuesta_langchain(resultado, tipo_analisis)
        
        return {
            "analisis": analisis_parseado,
            "respuesta_cruda": resultado,
            "tiempo_procesamiento": tiempo_procesamiento,
            "modelo_usado": f"{LLM_PROVIDER}",
            "tipo_analisis": tipo_analisis,
            "dominio": dominio,
            "exito": True
        }
        
    except Exception as e:
        return {
            "error": f"Error en análisis LangChain: {str(e)}",
            "tiempo_procesamiento": time.time() - tiempo_inicio,
            "exito": False
        }

def parsear_respuesta_langchain(respuesta, tipo_analisis):
    """Parsea la respuesta de LangChain en estructura"""
    try:
        resultado = {}
        
        # Buscar puntuación de similitud
        patrones_score = [
            r'(?:similitud|score|puntuación).*?(\d+)',
            r'(\d+)(?:/100|\s*%)',
            r'PUNTUACIÓN.*?(\d+)',
            r'Similitud general.*?(\d+)'
        ]
        
        for patron in patrones_score:
            match = re.search(patron, respuesta, re.IGNORECASE)
            if match:
                resultado['puntuacion_similitud'] = int(match.group(1))
                break
        
        if tipo_analisis == "comprehensive":
            # Extraer secciones estructuradas para análisis comprehensivo
            secciones = {
                'similitudes': r'\*\*SIMILITUDES.*?\*\*(.*?)(?=\*\*|$)',
                'diferencias': r'\*\*DIFERENCIAS.*?\*\*(.*?)(?=\*\*|$)',
                'temas_comunes': r'\*\*TEMAS COMUNES.*?\*\*(.*?)(?=\*\*|$)',
                'tipo_documentos': r'\*\*TIPO DE DOCUMENTOS.*?\*\*(.*?)(?=\*\*|$)',
                'analisis_estructura': r'\*\*ANÁLISIS DE ESTRUCTURA.*?\*\*(.*?)(?=\*\*|$)',
                'recomendaciones': r'\*\*RECOMENDACIONES.*?\*\*(.*?)(?=\*\*|$)',
                'resumen': r'\*\*RESUMEN.*?\*\*(.*?)(?=\*\*|$)'
            }
            
            for seccion, patron in secciones.items():
                match = re.search(patron, respuesta, re.DOTALL | re.IGNORECASE)
                if match:
                    contenido = match.group(1).strip()
                    contenido = re.sub(r'^\s*-\s*', '• ', contenido, flags=re.MULTILINE)
                    resultado[seccion] = contenido
        
        elif tipo_analisis == "quick":
            # Extraer información específica para análisis rápido
            resultado['analisis_rapido'] = respuesta
        
        # Si no se encontró estructura, usar respuesta completa
        if len(resultado) <= 1:
            resultado['analisis_completo'] = respuesta
        
        return resultado
        
    except Exception as e:
        return {
            'respuesta_cruda': respuesta,
            'error_parseo': str(e)
        }

print("✅ BLOQUE 9 - Funciones de análisis LangChain creadas")

## 📈 BLOQUE 10: Análisis TF-IDF

In [ ]:
def obtener_palabras_importantes(puntuaciones_tfidf, nombres_caracteristicas, n=10):
    """Obtiene las N palabras más importantes por TF-IDF"""
    indices_top = np.argsort(puntuaciones_tfidf)[-n:][::-1]
    return [(nombres_caracteristicas[i], float(puntuaciones_tfidf[i])) 
            for i in indices_top if puntuaciones_tfidf[i] > 0]

def analisis_tfidf(texto1, texto2, stop_words):
    """
    Realiza análisis usando TF-IDF
    
    Args:
        texto1, texto2: Textos a comparar
        stop_words: Set de palabras vacías
        
    Returns:
        dict: Resultados del análisis TF-IDF
    """
    if not texto1 or not texto2:
        return {
            "error": "Uno o ambos textos están vacíos",
            "similitud_tfidf": 0.0
        }
    
    try:
        vectorizador = TfidfVectorizer(
            stop_words=list(stop_words),
            max_features=1000,
            ngram_range=(1, 2),
            min_df=1
        )
        
        matriz_tfidf = vectorizador.fit_transform([texto1, texto2])
        similitud = cosine_similarity(matriz_tfidf[0:1], matriz_tfidf[1:2])[0][0]
        
        # Obtener palabras clave más importantes
        nombres_caracteristicas = vectorizador.get_feature_names_out()
        puntuaciones_tfidf = matriz_tfidf.toarray()
        
        palabras_top_doc1 = obtener_palabras_importantes(
            puntuaciones_tfidf[0], nombres_caracteristicas, 10
        )
        palabras_top_doc2 = obtener_palabras_importantes(
            puntuaciones_tfidf[1], nombres_caracteristicas, 10
        )
        
        return {
            "similitud_tfidf": float(similitud),
            "palabras_importantes_doc1": palabras_top_doc1,
            "palabras_importantes_doc2": palabras_top_doc2,
            "tamaño_vocabulario": len(nombres_caracteristicas),
            "porcentaje_similitud": float(similitud) * 100
        }
        
    except Exception as e:
        return {
            "error": f"Error en análisis TF-IDF: {str(e)}",
            "similitud_tfidf": 0.0
        }

print("✅ BLOQUE 10 - Funciones de análisis TF-IDF creadas")


## 🎯 BLOQUE 11: Función Principal de Comparación

In [ ]:
def comparar_pdfs_completo(ruta_pdf1, ruta_pdf2, usar_langchain=True, tipo_analisis="comprehensive", dominio=None):
    """
    Función principal que ejecuta todos los análisis de comparación
    
    Args:
        ruta_pdf1: Ruta al primer PDF
        ruta_pdf2: Ruta al segundo PDF
        usar_langchain: Si usar análisis con LangChain
        tipo_analisis: Tipo de análisis LangChain ("comprehensive", "quick", "domain")
        dominio: Dominio específico para análisis especializado
        
    Returns:
        dict: Resultados completos del análisis
    """
    print("🚀 Iniciando comparación completa de PDFs con LangChain...")
    tiempo_inicio = time.time()
    
    # Extraer texto de PDFs
    print("📄 Extrayendo texto del primer PDF...")
    datos_pdf1 = extraer_texto_pdf(ruta_pdf1)
    
    print("📄 Extrayendo texto del segundo PDF...")
    datos_pdf2 = extraer_texto_pdf(ruta_pdf2)
    
    # Verificar extracción exitosa
    if not datos_pdf1.get("exito") or not datos_pdf2.get("exito"):
        return {
            "error": "Error extrayendo texto de uno o ambos PDFs",
            "error_pdf1": datos_pdf1.get("error"),
            "error_pdf2": datos_pdf2.get("error")
        }
    
    # Limpiar textos
    texto1 = limpiar_texto(datos_pdf1["texto"])
    texto2 = limpiar_texto(datos_pdf2["texto"])
    
    if not texto1 or not texto2:
        return {
            "error": "Uno o ambos PDFs no contienen texto extraíble",
            "longitud_texto1": len(texto1),
            "longitud_texto2": len(texto2)
        }
    
    # Inicializar resultados
    resultados = {
        "informacion_pdfs": {
            "pdf1": {
                "nombre_archivo": datos_pdf1["nombre_archivo"],
                "paginas": datos_pdf1["paginas"],
                "tamaño_archivo": datos_pdf1["tamaño_archivo"],
                "estadisticas_texto": obtener_estadisticas_texto(texto1),
                "metadatos": extraer_metadatos_pdf(ruta_pdf1)
            },
            "pdf2": {
                "nombre_archivo": datos_pdf2["nombre_archivo"],
                "paginas": datos_pdf2["paginas"],
                "tamaño_archivo": datos_pdf2["tamaño_archivo"],
                "estadisticas_texto": obtener_estadisticas_texto(texto2),
                "metadatos": extraer_metadatos_pdf(ruta_pdf2)
            }
        },
        "configuracion": {
            "llm_provider": LLM_PROVIDER,
            "tipo_analisis": tipo_analisis,
            "dominio": dominio,
            "langchain_disponible": llm_disponible
        },
        "timestamp_analisis": time.strftime("%Y-%m-%d %H:%M:%S")
    }
    
    # Realizar análisis tradicionales
    print("📊 Ejecutando análisis básico...")
    resultados["analisis_basico"] = analisis_basico_difflib(texto1, texto2)
    
    print("🧠 Ejecutando análisis semántico...")
    resultados["analisis_semantico"] = analisis_semantico(texto1, texto2, modelo_embeddings)
    
    print("📈 Ejecutando análisis TF-IDF...")
    resultados["analisis_tfidf"] = analisis_tfidf(texto1, texto2, stop_words)
    
    # Análisis con LangChain si está disponible
    if usar_langchain and llm_disponible:
        print("🤖 Ejecutando análisis con LangChain...")
        resultados["analisis_langchain"] = analisis_langchain(texto1, texto2, tipo_analisis, dominio)
    
    # Calcular puntuación combinada
    puntuaciones = []
    
    if "analisis_basico" in resultados:
        puntuaciones.append(resultados["analisis_basico"]["ratio_similitud"])
    
    if "analisis_semantico" in resultados:
        puntuaciones.append(resultados["analisis_semantico"]["similitud_documento"])
    
    if "analisis_tfidf" in resultados:
        puntuaciones.append(resultados["analisis_tfidf"]["similitud_tfidf"])
    
    # Agregar puntuación LangChain si está disponible
    if usar_langchain and "analisis_langchain" in resultados and resultados["analisis_langchain"].get("exito"):
        puntuacion_langchain = resultados["analisis_langchain"]["analisis"].get("puntuacion_similitud")
        if puntuacion_langchain:
            puntuaciones.append(puntuacion_langchain / 100)
    
    resultados["puntuacion_similitud_combinada"] = np.mean(puntuaciones) if puntuaciones else 0.0
    resultados["tiempo_total_procesamiento"] = time.time() - tiempo_inicio
    
    print(f"✅ Análisis completado en {resultados['tiempo_total_procesamiento']:.2f} segundos")
    return resultados

print("✅ BLOQUE 11 - Función principal de comparación creada")

## 📊 BLOQUE 12: Funciones de Visualización Actualizadas

In [ ]:
def mostrar_informacion_pdfs(resultados):
    """Muestra información básica de los PDFs"""
    if "error" in resultados:
        display(HTML(f"""
        <div style="background-color: #ffebee; border-left: 5px solid #f44336; padding: 10px; margin: 10px 0;">
            <h3 style="color: #d32f2f;">❌ Error en el análisis</h3>
            <p><strong>Error:</strong> {resultados['error']}</p>
        </div>
        """))
        return
    
    info_pdfs = resultados.get("informacion_pdfs", {})
    pdf1 = info_pdfs.get("pdf1", {})
    pdf2 = info_pdfs.get("pdf2", {})
    config = resultados.get("configuracion", {})
    
    display(HTML(f"""
    <div style="background-color: #e3f2fd; border-left: 5px solid #2196f3; padding: 15px; margin: 10px 0;">
        <h2 style="color: #1565c0; margin-top: 0;">📊 Información de los Documentos</h2>
        <p style="color: #1565c0;"><strong>🤖 LLM Provider:</strong> {config.get('llm_provider', 'N/A').upper()}</p>
        <p style="color: #1565c0;"><strong>🔧 Tipo de análisis:</strong> {config.get('tipo_analisis', 'N/A')}</p>
    </div>
    
    <table style="border-collapse: collapse; width: 100%; border: 1px solid #ddd; margin: 20px 0;">
        <thead style="background-color: #f5f5f5;">
            <tr>
                <th style="border: 1px solid #ddd; padding: 12px; text-align: left;">Atributo</th>
                <th style="border: 1px solid #ddd; padding: 12px; text-align: left;">PDF 1</th>
                <th style="border: 1px solid #ddd; padding: 12px; text-align: left;">PDF 2</th>
            </tr>
        </thead>
        <tbody>
            <tr>
                <td style="border: 1px solid #ddd; padding: 8px;"><strong>Archivo</strong></td>
                <td style="border: 1px solid #ddd; padding: 8px;">{pdf1.get('nombre_archivo', 'N/A')}</td>
                <td style="border: 1px solid #ddd; padding: 8px;">{pdf2.get('nombre_archivo', 'N/A')}</td>
            </tr>
            <tr>
                <td style="border: 1px solid #ddd; padding: 8px;"><strong>Páginas</strong></td>
                <td style="border: 1px solid #ddd; padding: 8px;">{pdf1.get('paginas', 'N/A')}</td>
                <td style="border: 1px solid #ddd; padding: 8px;">{pdf2.get('paginas', 'N/A')}</td>
            </tr>
            <tr>
                <td style="border: 1px solid #ddd; padding: 8px;"><strong>Palabras</strong></td>
                <td style="border: 1px solid #ddd; padding: 8px;">{pdf1.get('estadisticas_texto', {}).get('palabras', 'N/A')}</td>
                <td style="border: 1px solid #ddd; padding: 8px;">{pdf2.get('estadisticas_texto', {}).get('palabras', 'N/A')}</td>
            </tr>
            <tr>
                <td style="border: 1px solid #ddd; padding: 8px;"><strong>Oraciones</strong></td>
                <td style="border: 1px solid #ddd; padding: 8px;">{pdf1.get('estadisticas_texto', {}).get('oraciones', 'N/A')}</td>
                <td style="border: 1px solid #ddd; padding: 8px;">{pdf2.get('estadisticas_texto', {}).get('oraciones', 'N/A')}</td>
            </tr>
            <tr>
                <td style="border: 1px solid #ddd; padding: 8px;"><strong>Tamaño archivo</strong></td>
                <td style="border: 1px solid #ddd; padding: 8px;">{pdf1.get('tamaño_archivo', 0) / 1024:.1f} KB</td>
                <td style="border: 1px solid #ddd; padding: 8px;">{pdf2.get('tamaño_archivo', 0) / 1024:.1f} KB</td>
            </tr>
        </tbody>
    </table>
    """))

def mostrar_resultados_similitud(resultados):
    """Muestra los resultados de similitud de todos los análisis"""
    if "error" in resultados:
        return
    
    # Extraer puntuaciones
    puntuacion_combinada = resultados.get("puntuacion_similitud_combinada", 0)
    basico = resultados.get("analisis_basico", {}).get("porcentaje_similitud", 0)
    semantico = resultados.get("analisis_semantico", {}).get("porcentaje_similitud", 0)
    tfidf = resultados.get("analisis_tfidf", {}).get("porcentaje_similitud", 0)
    
    # LangChain si está disponible
    langchain_score = 0
    if "analisis_langchain" in resultados and resultados["analisis_langchain"].get("exito"):
        langchain_score = resultados["analisis_langchain"]["analisis"].get("puntuacion_similitud", 0)
    
    provider = resultados.get("configuracion", {}).get("llm_provider", "N/A").upper()
    
    display(HTML(f"""
    <div style="background-color: #f3e5f5; border-left: 5px solid #9c27b0; padding: 15px; margin: 20px 0;">
        <h2 style="color: #7b1fa2; margin-top: 0;">🎯 Puntuaciones de Similitud</h2>
        
        <div style="background-color: #ffffff; padding: 20px; border-radius: 8px; margin: 15px 0;">
            <h3 style="color: #4a148c;">📊 Puntuación Combinada: {puntuacion_combinada*100:.1f}%</h3>
            <div style="background-color: #e0e0e0; border-radius: 10px; overflow: hidden; margin: 10px 0;">
                <div style="background-color: #9c27b0; height: 20px; width: {puntuacion_combinada*100:.1f}%; border-radius: 10px;"></div>
            </div>
        </div>
        
        <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 15px; margin-top: 20px;">
            <div style="background-color: #ffffff; padding: 15px; border-radius: 8px;">
                <h4 style="color: #1976d2; margin-top: 0;">📋 Análisis Básico</h4>
                <p style="font-size: 24px; font-weight: bold; color: #1976d2; margin: 10px 0;">{basico:.1f}%</p>
                <div style="background-color: #e3f2fd; height: 8px; border-radius: 4px;">
                    <div style="background-color: #2196f3; height: 8px; width: {basico:.1f}%; border-radius: 4px;"></div>
                </div>
            </div>
            
            <div style="background-color: #ffffff; padding: 15px; border-radius: 8px;">
                <h4 style="color: #388e3c; margin-top: 0;">🧠 Análisis Semántico</h4>
                <p style="font-size: 24px; font-weight: bold; color: #388e3c; margin: 10px 0;">{semantico:.1f}%</p>
                <div style="background-color: #e8f5e8; height: 8px; border-radius: 4px;">
                    <div style="background-color: #4caf50; height: 8px; width: {semantico:.1f}%; border-radius: 4px;"></div>
                </div>
            </div>
            
            <div style="background-color: #ffffff; padding: 15px; border-radius: 8px;">
                <h4 style="color: #f57c00; margin-top: 0;">📈 Análisis TF-IDF</h4>
                <p style="font-size: 24px; font-weight: bold; color: #f57c00; margin: 10px 0;">{tfidf:.1f}%</p>
                <div style="background-color: #fff3e0; height: 8px; border-radius: 4px;">
                    <div style="background-color: #ff9800; height: 8px; width: {tfidf:.1f}%; border-radius: 4px;"></div>
                </div>
            </div>
            
            <div style="background-color: #ffffff; padding: 15px; border-radius: 8px;">
                <h4 style="color: #d32f2f; margin-top: 0;">🤖 {provider} LangChain</h4>
                <p style="font-size: 24px; font-weight: bold; color: #d32f2f; margin: 10px 0;">{langchain_score:.1f}%</p>
                <div style="background-color: #ffebee; height: 8px; border-radius: 4px;">
                    <div style="background-color: #f44336; height: 8px; width: {langchain_score:.1f}%; border-radius: 4px;"></div>
                </div>
            </div>
        </div>
    </div>
    """))

def mostrar_analisis_langchain(resultados):
    """Muestra los resultados del análisis LangChain de forma estructurada"""
    if "analisis_langchain" not in resultados or not resultados["analisis_langchain"].get("exito"):
        return
    
    analisis_data = resultados["analisis_langchain"]
    analisis = analisis_data["analisis"]
    tiempo = analisis_data["tiempo_procesamiento"]
    provider = resultados.get("configuracion", {}).get("llm_provider", "Unknown").upper()
    tipo = analisis_data.get("tipo_analisis", "comprehensive")
    dominio = analisis_data.get("dominio")
    
    display(HTML(f"""
    <div style="background-color: #fce4ec; border-left: 5px solid #e91e63; padding: 15px; margin: 20px 0;">
        <h2 style="color: #c2185b; margin-top: 0;">🤖 Análisis Inteligente con LangChain</h2>
        <p style="color: #880e4f; margin: 5px 0;"><strong>Provider:</strong> {provider}</p>
        <p style="color: #880e4f; margin: 5px 0;"><strong>Tipo de análisis:</strong> {tipo}</p>
        {f'<p style="color: #880e4f; margin: 5px 0;"><strong>Dominio:</strong> {dominio}</p>' if dominio else ''}
        <p style="color: #880e4f; margin: 5px 0;"><strong>Tiempo de procesamiento:</strong> {tiempo:.2f} segundos</p>
    </div>
    """))
    
    # Mostrar secciones del análisis según el tipo
    if tipo == "comprehensive":
        # Mostrar análisis comprehensivo estructurado
        if "similitudes" in analisis:
            display(HTML(f"""
            <div style="background-color: #e8f5e8; padding: 15px; margin: 10px 0; border-radius: 8px;">
                <h3 style="color: #2e7d32; margin-top: 0;">✅ Similitudes Principales</h3>
                <div style="white-space: pre-line; line-height: 1.6;">{analisis['similitudes']}</div>
            </div>
            """))
        
        if "diferencias" in analisis:
            display(HTML(f"""
            <div style="background-color: #fff3e0; padding: 15px; margin: 10px 0; border-radius: 8px;">
                <h3 style="color: #ef6c00; margin-top: 0;">❗ Diferencias Principales</h3>
                <div style="white-space: pre-line; line-height: 1.6;">{analisis['diferencias']}</div>
            </div>
            """))
        
        if "temas_comunes" in analisis:
            display(HTML(f"""
            <div style="background-color: #e3f2fd; padding: 15px; margin: 10px 0; border-radius: 8px;">
                <h3 style="color: #1565c0; margin-top: 0;">🎯 Temas Comunes</h3>
                <div style="white-space: pre-line; line-height: 1.6;">{analisis['temas_comunes']}</div>
            </div>
            """))
        
        if "recomendaciones" in analisis:
            display(HTML(f"""
            <div style="background-color: #f1f8e9; padding: 15px; margin: 10px 0; border-radius: 8px;">
                <h3 style="color: #33691e; margin-top: 0;">💡 Recomendaciones</h3>
                <div style="white-space: pre-line; line-height: 1.6;">{analisis['recomendaciones']}</div>
            </div>
            """))
        
        if "resumen" in analisis:
            display(HTML(f"""
            <div style="background-color: #f3e5f5; padding: 15px; margin: 10px 0; border-radius: 8px;">
                <h3 style="color: #7b1fa2; margin-top: 0;">📝 Resumen Ejecutivo</h3>
                <div style="white-space: pre-line; line-height: 1.6; font-style: italic;">{analisis['resumen']}</div>
            </div>
            """))
    
    else:
        # Mostrar análisis rápido o de dominio
        content_key = "analisis_rapido" if "analisis_rapido" in analisis else "analisis_completo"
        if content_key in analisis:
            display(HTML(f"""
            <div style="background-color: #f5f5f5; padding: 15px; margin: 10px 0; border-radius: 8px;">
                <h3 style="color: #424242; margin-top: 0;">📄 Análisis {tipo.title()}</h3>
                <div style="white-space: pre-line; line-height: 1.6;">{analisis[content_key]}</div>
            </div>
            """))

def crear_grafico_comparacion(resultados):
    """Crea un gráfico de barras con las puntuaciones de similitud incluyendo LangChain"""
    if "error" in resultados:
        return
    
    # Extraer datos
    metodos = []
    puntuaciones = []
    
    if "analisis_basico" in resultados:
        metodos.append("Análisis\nBásico")
        puntuaciones.append(resultados["analisis_basico"]["porcentaje_similitud"])
    
    if "analisis_semantico" in resultados:
        metodos.append("Análisis\nSemántico") 
        puntuaciones.append(resultados["analisis_semantico"]["porcentaje_similitud"])
    
    if "analisis_tfidf" in resultados:
        metodos.append("TF-IDF")
        puntuaciones.append(resultados["analisis_tfidf"]["porcentaje_similitud"])
    
    if "analisis_langchain" in resultados and resultados["analisis_langchain"].get("exito"):
        provider = resultados.get("configuracion", {}).get("llm_provider", "LLM").upper()
        metodos.append(f"{provider}\nLangChain")
        puntuaciones.append(resultados["analisis_langchain"]["analisis"].get("puntuacion_similitud", 0))
    
    # Agregar puntuación combinada
    metodos.append("Combinada")
    puntuaciones.append(resultados["puntuacion_similitud_combinada"] * 100)
    
    # Crear gráfico
    plt.figure(figsize=(12, 6))
    colores = ['#2196f3', '#4caf50', '#ff9800', '#f44336', '#9c27b0']
    
    barras = plt.bar(metodos, puntuaciones, color=colores[:len(metodos)], alpha=0.8)
    
    # Agregar valores en las barras
    for barra, puntuacion in zip(barras, puntuaciones):
        plt.text(barra.get_x() + barra.get_width()/2, barra.get_height() + 1, 
                f'{puntuacion:.1f}%', ha='center', va='bottom', fontweight='bold')
    
    plt.title('Comparación de Puntuaciones de Similitud (con LangChain)', fontsize=16, fontweight='bold', pad=20)
    plt.ylabel('Similitud (%)', fontsize=12)
    plt.xlabel('Método de Análisis', fontsize=12)
    plt.ylim(0, 105)
    plt.grid(axis='y', alpha=0.3)
    
    # Línea de referencia en 50%
    plt.axhline(y=50, color='red', linestyle='--', alpha=0.5, label='50% (referencia)')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

def mostrar_palabras_importantes(resultados):
    """Muestra las palabras más importantes de cada documento según TF-IDF"""
    if "analisis_tfidf" not in resultados or "error" in resultados["analisis_tfidf"]:
        return
    
    tfidf_data = resultados["analisis_tfidf"]
    palabras_doc1 = tfidf_data.get("palabras_importantes_doc1", [])
    palabras_doc2 = tfidf_data.get("palabras_importantes_doc2", [])
    
    display(HTML("""
    <div style="background-color: #fff8e1; border-left: 5px solid #ffc107; padding: 15px; margin: 20px 0;">
        <h2 style="color: #f57c00; margin-top: 0;">📝 Palabras Más Importantes (TF-IDF)</h2>
    </div>
    """))
    
    # Crear tabla comparativa
    html_table = """
    <table style="border-collapse: collapse; width: 100%; margin: 20px 0;">
        <thead style="background-color: #f5f5f5;">
            <tr>
                <th style="border: 1px solid #ddd; padding: 12px; text-align: left;">PDF 1 - Palabras Clave</th>
                <th style="border: 1px solid #ddd; padding: 12px; text-align: left;">PDF 2 - Palabras Clave</th>
            </tr>
        </thead>
        <tbody>
    """
    
    max_len = max(len(palabras_doc1), len(palabras_doc2))
    
    for i in range(max_len):
        palabra1 = palabras_doc1[i] if i < len(palabras_doc1) else ("", 0)
        palabra2 = palabras_doc2[i] if i < len(palabras_doc2) else ("", 0)
        
        html_table += f"""
            <tr>
                <td style="border: 1px solid #ddd; padding: 8px;">
                    {palabra1[0]} <span style="color: #666; font-size: 0.9em;">({palabra1[1]:.3f})</span>
                </td>
                <td style="border: 1px solid #ddd; padding: 8px;">
                    {palabra2[0]} <span style="color: #666; font-size: 0.9em;">({palabra2[1]:.3f})</span>
                </td>
            </tr>
        """
    
    html_table += "</tbody></table>"
    display(HTML(html_table))

print("✅ BLOQUE 12 - Funciones de visualización creadas")


## 🚀 BLOQUE 13: Funciones de Ejecución Principales

In [ ]:

def ejecutar_comparacion_completa():
    """Ejecuta todo el proceso de comparación con LangChain y muestra resultados"""
    
    # Verificar que los archivos estén configurados y existan
    if not archivos_ok:
        print("❌ Por favor, configura las rutas de los PDFs en el BLOQUE 2")
        return None
    
    print("🚀 Iniciando comparación completa con LangChain...")
    print(f"📄 PDF 1: {PDF1_PATH}")
    print(f"📄 PDF 2: {PDF2_PATH}")
    print(f"🤖 Provider LLM: {LLM_PROVIDER}")
    print("-" * 60)
    
    # Ejecutar comparación
    resultados = comparar_pdfs_completo(
        PDF1_PATH, 
        PDF2_PATH, 
        usar_langchain=llm_disponible,
        tipo_analisis="comprehensive"
    )
    
    # Mostrar resultados
    if "error" not in resultados:
        print("\n🎉 ¡Comparación completada exitosamente!")
        print(f"⏱️ Tiempo total: {resultados['tiempo_total_procesamiento']:.2f} segundos")
        print(f"🎯 Similitud combinada: {resultados['puntuacion_similitud_combinada']*100:.1f}%")
        
        # Mostrar visualizaciones
        mostrar_informacion_pdfs(resultados)
        mostrar_resultados_similitud(resultados)
        crear_grafico_comparacion(resultados)
        mostrar_palabras_importantes(resultados)
        mostrar_analisis_langchain(resultados)
        
    else:
        print(f"❌ Error en la comparación: {resultados['error']}")
    
    return resultados

def comparacion_rapida_langchain():
    """Ejecuta comparación rápida usando LangChain"""
    if not archivos_ok or not llm_disponible:
        print("❌ Verifica PDFs y conexión LLM")
        return None
    
    print("⚡ Ejecutando comparación rápida con LangChain...")
    
    resultados = comparar_pdfs_completo(
        PDF1_PATH, 
        PDF2_PATH, 
        usar_langchain=True,
        tipo_analisis="quick"
    )
    
    if "error" not in resultados:
        combinada = resultados["puntuacion_similitud_combinada"] * 100
        langchain_data = resultados.get("analisis_langchain", {})
        
        print(f"""
📊 RESULTADOS RÁPIDOS:
   • Similitud combinada: {combinada:.1f}%
   • Tiempo total: {resultados['tiempo_total_procesamiento']:.2f}s
   • Provider LLM: {LLM_PROVIDER.upper()}
   • Tiempo LangChain: {langchain_data.get('tiempo_procesamiento', 0):.2f}s
        """)
        
        # Mostrar análisis rápido de LangChain
        if langchain_data.get("exito"):
            print("\n🤖 ANÁLISIS RÁPIDO LANGCHAIN:")
            respuesta = langchain_data.get("respuesta_cruda", "")
            print(respuesta[:500] + "..." if len(respuesta) > 500 else respuesta)
    
    return resultados

def analisis_por_dominio(dominio):
    """Ejecuta análisis especializado por dominio específico"""
    if not archivos_ok or not llm_disponible:
        print("❌ Verifica PDFs y conexión LLM")
        return None
    
    print(f"🎯 Ejecutando análisis especializado en: {dominio}")
    
    resultados = comparar_pdfs_completo(
        PDF1_PATH, 
        PDF2_PATH, 
        usar_langchain=True,
        tipo_analisis="domain",
        dominio=dominio
    )
    
    if "error" not in resultados:
        print("✅ Análisis por dominio completado")
        mostrar_analisis_langchain(resultados)
    
    return resultados

print("✅ BLOQUE 13 - Funciones de ejecución principales creadas")

## 🛠️ BLOQUE 14: Funciones de Utilidad Adicionales

In [ ]:
def cambiar_proveedor_llm(nuevo_proveedor):
    """
    Cambia el proveedor LLM y reinicializa el modelo
    
    Args:
        nuevo_proveedor: "vllm", "openai", o "ollama"
    """
    global LLM_PROVIDER, llm_model, llm_disponible
    
    print(f"🔄 Cambiando proveedor LLM de {LLM_PROVIDER} a {nuevo_proveedor}")
    
    LLM_PROVIDER = nuevo_proveedor
    llm_model = inicializar_llm(nuevo_proveedor)
    llm_disponible = llm_model is not None
    
    if llm_disponible:
        print(f"✅ Proveedor cambiado exitosamente a {nuevo_proveedor}")
    else:
        print(f"❌ Error cambiando a {nuevo_proveedor}")

def probar_conexion_llm():
    """Prueba la conexión con el LLM actual"""
    if not llm_disponible:
        print("❌ No hay LLM disponible")
        return False
    
    print(f"🧪 Probando conexión con {LLM_PROVIDER}...")
    
    try:
        # Crear una prueba simple
        test_chain = LLMChain(
            llm=llm_model,
            prompt=PromptTemplate(
                input_variables=["test"],
                template="Responde brevemente: {test}"
            )
        )
        resultado = test_chain.run({"test": "¿Funciona la conexión?"})
        
        print(f"✅ Conexión exitosa. Respuesta: {resultado[:100]}...")
        return True
        
    except Exception as e:
        print(f"❌ Error probando conexión: {str(e)}")
        return False

def exportar_resultados(resultados, nombre_archivo="comparacion_resultados.json"):
    """Exporta los resultados a un archivo JSON"""
    try:
        with open(nombre_archivo, 'w', encoding='utf-8') as f:
            json.dump(resultados, f, indent=2, ensure_ascii=False, default=str)
        print(f"✅ Resultados exportados a {nombre_archivo}")
    except Exception as e:
        print(f"❌ Error exportando resultados: {str(e)}")

def obtener_estadisticas_comparacion(resultados):
    """Obtiene estadísticas resumidas de la comparación"""
    if "error" in resultados:
        return {"error": "No se pueden obtener estadísticas de resultados con error"}
    
    stats = {
        "resumen_general": {
            "similitud_combinada": f"{resultados.get('puntuacion_similitud_combinada', 0)*100:.1f}%",
            "tiempo_total": f"{resultados.get('tiempo_total_procesamiento', 0):.2f}s",
            "proveedor_llm": resultados.get("configuracion", {}).get("llm_provider", "N/A"),
            "analisis_exitosos": 0
        },
        "puntuaciones_individuales": {},
        "informacion_documentos": {}
    }
    
    # Contar análisis exitosos y recopilar puntuaciones
    if "analisis_basico" in resultados and "error" not in resultados["analisis_basico"]:
        stats["resumen_general"]["analisis_exitosos"] += 1
        stats["puntuaciones_individuales"]["basico"] = f"{resultados['analisis_basico']['porcentaje_similitud']:.1f}%"
    
    if "analisis_semantico" in resultados and "error" not in resultados["analisis_semantico"]:
        stats["resumen_general"]["analisis_exitosos"] += 1
        stats["puntuaciones_individuales"]["semantico"] = f"{resultados['analisis_semantico']['porcentaje_similitud']:.1f}%"
    
    if "analisis_tfidf" in resultados and "error" not in resultados["analisis_tfidf"]:
        stats["resumen_general"]["analisis_exitosos"] += 1
        stats["puntuaciones_individuales"]["tfidf"] = f"{resultados['analisis_tfidf']['porcentaje_similitud']:.1f}%"
    
    if "analisis_langchain" in resultados and resultados["analisis_langchain"].get("exito"):
        stats["resumen_general"]["analisis_exitosos"] += 1
        langchain_score = resultados["analisis_langchain"]["analisis"].get("puntuacion_similitud", 0)
        stats["puntuaciones_individuales"]["langchain"] = f"{langchain_score:.1f}%"
    
    # Información de documentos
    info_pdfs = resultados.get("informacion_pdfs", {})
    if info_pdfs:
        pdf1 = info_pdfs.get("pdf1", {})
        pdf2 = info_pdfs.get("pdf2", {})
        
        stats["informacion_documentos"] = {
            "pdf1": {
                "nombre": pdf1.get("nombre_archivo", "N/A"),
                "paginas": pdf1.get("paginas", 0),
                "palabras": pdf1.get("estadisticas_texto", {}).get("palabras", 0)
            },
            "pdf2": {
                "nombre": pdf2.get("nombre_archivo", "N/A"),
                "paginas": pdf2.get("paginas", 0),
                "palabras": pdf2.get("estadisticas_texto", {}).get("palabras", 0)
            }
        }
    
    return stats

def generar_reporte_texto(resultados):
    """Genera un reporte en texto plano de los resultados"""
    if "error" in resultados:
        return f"ERROR: {resultados['error']}"
    
    stats = obtener_estadisticas_comparacion(resultados)
    
    reporte = []
    reporte.append("=" * 60)
    reporte.append("REPORTE DE COMPARACIÓN DE DOCUMENTOS PDF")
    reporte.append("=" * 60)
    
    # Información general
    reporte.append(f"\n📊 RESUMEN GENERAL:")
    reporte.append(f"   • Similitud combinada: {stats['resumen_general']['similitud_combinada']}")
    reporte.append(f"   • Tiempo total: {stats['resumen_general']['tiempo_total']}")
    reporte.append(f"   • Proveedor LLM: {stats['resumen_general']['proveedor_llm']}")
    reporte.append(f"   • Análisis exitosos: {stats['resumen_general']['analisis_exitosos']}/4")
    
    # Información de documentos
    reporte.append(f"\n📄 DOCUMENTOS ANALIZADOS:")
    doc_info = stats["informacion_documentos"]
    reporte.append(f"   • PDF 1: {doc_info['pdf1']['nombre']} ({doc_info['pdf1']['paginas']} páginas, {doc_info['pdf1']['palabras']} palabras)")
    reporte.append(f"   • PDF 2: {doc_info['pdf2']['nombre']} ({doc_info['pdf2']['paginas']} páginas, {doc_info['pdf2']['palabras']} palabras)")
    
    # Puntuaciones individuales
    reporte.append(f"\n🎯 PUNTUACIONES POR MÉTODO:")
    puntuaciones = stats["puntuaciones_individuales"]
    if "basico" in puntuaciones:
        reporte.append(f"   • Análisis básico (difflib): {puntuaciones['basico']}")
    if "semantico" in puntuaciones:
        reporte.append(f"   • Análisis semántico: {puntuaciones['semantico']}")
    if "tfidf" in puntuaciones:
        reporte.append(f"   • Análisis TF-IDF: {puntuaciones['tfidf']}")
    if "langchain" in puntuaciones:
        reporte.append(f"   • Análisis LangChain: {puntuaciones['langchain']}")
    
    # Análisis LangChain detallado si está disponible
    if "analisis_langchain" in resultados and resultados["analisis_langchain"].get("exito"):
        langchain_data = resultados["analisis_langchain"]
        reporte.append(f"\n🤖 ANÁLISIS LANGCHAIN DETALLADO:")
        reporte.append(f"   • Tiempo de procesamiento: {langchain_data['tiempo_procesamiento']:.2f}s")
        reporte.append(f"   • Tipo de análisis: {langchain_data.get('tipo_analisis', 'N/A')}")
        
        analisis = langchain_data.get("analisis", {})
        if "similitudes" in analisis:
            reporte.append(f"   • Similitudes principales:")
            similitudes = analisis["similitudes"].replace("•", "     -")
            reporte.append(f"     {similitudes[:200]}...")
        
        if "diferencias" in analisis:
            reporte.append(f"   • Diferencias principales:")
            diferencias = analisis["diferencias"].replace("•", "     -")
            reporte.append(f"     {diferencias[:200]}...")
    
    reporte.append("\n" + "=" * 60)
    reporte.append(f"Generado: {time.strftime('%Y-%m-%d %H:%M:%S')}")
    
    return "\n".join(reporte)

print("✅ BLOQUE 14 - Funciones de utilidad adicionales creadas")

## 📋 BLOQUE 15: Instrucciones de Uso

In [ ]:
def mostrar_instrucciones():
    """Muestra las instrucciones completas de uso"""
    print("""
🎯 INSTRUCCIONES DE USO CON LANGCHAIN:

1️⃣ CONFIGURAR PATHS (BLOQUE 2):
   Modifica las variables PDF1_PATH y PDF2_PATH con las rutas a tus documentos
   
   Configura el proveedor LLM deseado:
   - LLM_PROVIDER = "vllm"     # Para vLLM local
   - LLM_PROVIDER = "openai"   # Para OpenAI API
   - LLM_PROVIDER = "ollama"   # Para Ollama local

2️⃣ CONFIGURAR LLM:
   
   🔹 Para vLLM:
   python -m vllm.entrypoints.openai.api_server \\
     --model meta-llama/Llama-2-7b-chat-hf \\
     --host localhost \\
     --port 8000

   🔹 Para OpenAI:
   Configura tu API key en OPENAI_CONFIG["api_key"]

   🔹 Para Ollama:
   ollama serve
   ollama pull llama2

3️⃣ EJECUTAR COMPARACIÓN:
   • Comparación completa: 
     resultados = ejecutar_comparacion_completa()
   
   • Comparación rápida: 
     resultados = comparacion_rapida_langchain()
   
   • Análisis por dominio: 
     resultados = analisis_por_dominio("legal")

4️⃣ CAMBIAR PROVEEDOR EN TIEMPO REAL:
   cambiar_proveedor_llm("openai")  # Cambia a OpenAI
   cambiar_proveedor_llm("ollama")  # Cambia a Ollama

5️⃣ PROBAR CONEXIÓN:
   probar_conexion_llm()

6️⃣ UTILIDADES ADICIONALES:
   # Exportar resultados
   exportar_resultados(resultados, "mi_comparacion.json")
   
   # Obtener estadísticas
   stats = obtener_estadisticas_comparacion(resultados)
   
   # Generar reporte en texto
   reporte = generar_reporte_texto(resultados)
   print(reporte)
""")

def mostrar_modelos_recomendados():
    """Muestra los modelos recomendados para cada proveedor"""
    print("""
🔧 MODELOS RECOMENDADOS:

📌 vLLM:
   • meta-llama/Llama-2-7b-chat-hf (General, español/inglés)
   • mistralai/Mistral-7B-Instruct-v0.1 (Rápido, multilingüe)
   • clibrain/lince-zero-spanish-7b (Especializado en español)
   • microsoft/DialoGPT-medium-spanish (Conversacional en español)

📌 Ollama:
   • llama2, llama2:13b (General purpose)
   • mistral, mistral:7b (Eficiente y rápido)
   • codellama:7b (Especializado en código)
   • neural-chat:7b (Conversacional)

📌 OpenAI:
   • gpt-3.5-turbo (Rápido y económico)
   • gpt-4 (Máxima calidad y razonamiento)
   • gpt-4-turbo (Balance calidad/velocidad)

💡 VENTAJAS DE LANGCHAIN:
   ✅ Fácil cambio entre proveedores
   ✅ Templates de prompts reutilizables
   ✅ Manejo automático de errores
   ✅ Callbacks para debugging
   ✅ Chains para análisis complejos
   ✅ Análisis especializados por dominio
""")

# Mostrar instrucciones al cargar
mostrar_instrucciones()
mostrar_modelos_recomendados()
print("✅ BLOQUE 15 - Instrucciones mostradas")

## 🎮 BLOQUE 16: Ejemplos de Casos de Uso

In [ ]:
def ejemplos_casos_uso():
    """Muestra ejemplos prácticos de casos de uso"""
    
    print("""
🎮 EJEMPLOS DE CASOS DE USO:

1️⃣ COMPARACIÓN BÁSICA:
   # Configura rutas y ejecuta
   resultados = ejecutar_comparacion_completa()

2️⃣ ANÁLISIS RÁPIDO PARA DECISIÓN URGENTE:
   # Solo análisis esenciales
   resultados = comparacion_rapida_langchain()

3️⃣ ANÁLISIS ESPECIALIZADO POR DOMINIO:
   # Para documentos legales
   resultados_legal = analisis_por_dominio("documentos legales y contratos")
   
   # Para documentación técnica
   resultados_tech = analisis_por_dominio("documentación técnica de software")
   
   # Para papers académicos
   resultados_academic = analisis_por_dominio("papers científicos y académicos")

4️⃣ COMPARACIÓN CON MÚLTIPLES MODELOS:
   # Probar con vLLM
   cambiar_proveedor_llm("vllm")
   resultados_vllm = ejecutar_comparacion_completa()
   
   # Probar con OpenAI
   cambiar_proveedor_llm("openai")
   resultados_openai = ejecutar_comparacion_completa()
   
   # Comparar resultados
   print("vLLM:", resultados_vllm["puntuacion_similitud_combinada"])
   print("OpenAI:", resultados_openai["puntuacion_similitud_combinada"])

5️⃣ ANÁLISIS BATCH DE MÚLTIPLES DOCUMENTOS:
   documentos = [
       ("doc1.pdf", "doc2.pdf"),
       ("doc3.pdf", "doc4.pdf"),
       ("doc5.pdf", "doc6.pdf")
   ]
   
   resultados_batch = []
   for pdf1, pdf2 in documentos:
       resultado = comparar_pdfs_completo(pdf1, pdf2)
       resultados_batch.append(resultado)

6️⃣ EXPORTACIÓN Y REPORTING:
   # Ejecutar análisis
   resultados = ejecutar_comparacion_completa()
   
   # Exportar datos completos
   exportar_resultados(resultados, "analisis_completo.json")
   
   # Generar reporte ejecutivo
   reporte = generar_reporte_texto(resultados)
   with open("reporte_ejecutivo.txt", "w", encoding="utf-8") as f:
       f.write(reporte)
   
   # Obtener estadísticas clave
   stats = obtener_estadisticas_comparacion(resultados)
   print("Estadísticas:", stats)

7️⃣ DEBUGGING Y TROUBLESHOOTING:
   # Verificar conexiones
   probar_conexion_llm()
   
   # Cambiar modelo si hay problemas
   cambiar_proveedor_llm("ollama")
   
   # Análisis paso a paso
   datos1 = extraer_texto_pdf(PDF1_PATH)
   datos2 = extraer_texto_pdf(PDF2_PATH)
   print("Extracción exitosa:", datos1["exito"], datos2["exito"])
    """)

def configuracion_avanzada():
    """Muestra opciones de configuración avanzada"""
    
    print("""
⚙️ CONFIGURACIÓN AVANZADA:

1️⃣ PERSONALIZAR TEMPLATES DE PROMPTS:
   # Modificar templates existentes
   prompt_templates["comprehensive"].template = "Tu prompt personalizado..."
   
   # Crear template específico
   template_personalizado = PromptTemplate(
       input_variables=["documento1", "documento2"],
       template="Analiza desde perspectiva de: {perspectiva}..."
   )

2️⃣ AJUSTAR PARÁMETROS DE MODELOS:
   # Para vLLM
   VLLM_CONFIG["temperature"] = 0.1  # Más determinístico
   VLLM_CONFIG["max_tokens"] = 4096  # Respuestas más largas
   
   # Para OpenAI
   OPENAI_CONFIG["temperature"] = 0.7  # Más creativo
   OPENAI_CONFIG["model"] = "gpt-4"    # Modelo más potente

3️⃣ CONFIGURAR LÍMITES DE TEXTO:
   # Ajustar truncamiento para LLM
   texto_truncado = truncar_texto_para_llm(texto, max_chars=6000)

4️⃣ PERSONALIZAR ANÁLISIS SEMÁNTICO:
   # Usar modelo de embeddings diferente
   modelo_embeddings = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

5️⃣ CONFIGURAR PALABRAS VACÍAS PERSONALIZADAS:
   # Agregar palabras específicas del dominio
   stop_words_custom = stop_words.union({'palabra1', 'palabra2'})
    """)

# Mostrar ejemplos
ejemplos_casos_uso()
configuracion_avanzada()
print("✅ BLOQUE 16 - Ejemplos de casos de uso mostrados")

## 🚀 BLOQUE 17: Ejecución Final y Estado del Sistema

In [ ]:
def verificar_estado_sistema():
    """Verifica el estado completo del sistema"""
    print("🔍 VERIFICANDO ESTADO DEL SISTEMA:")
    print("=" * 50)
    
    # 1. Verificar archivos PDF
    print(f"📄 Archivos PDF configurados: {'✅' if archivos_ok else '❌'}")
    if archivos_ok:
        print(f"   • PDF 1: {PDF1_PATH}")
        print(f"   • PDF 2: {PDF2_PATH}")
    else:
        print("   • Configura PDF1_PATH y PDF2_PATH en BLOQUE 2")
    
    # 2. Verificar LLM
    print(f"🤖 LLM configurado: {'✅' if llm_disponible else '❌'}")
    print(f"   • Proveedor: {LLM_PROVIDER}")
    if llm_disponible:
        print("   • Estado: Conectado y listo")
    else:
        print("   • Estado: No disponible - verifica configuración")
    
    # 3. Verificar modelos de ML
    print(f"🧠 Modelo embeddings: {'✅' if 'modelo_embeddings' in globals() else '❌'}")
    print(f"📚 NLTK configurado: {'✅' if 'stop_words' in globals() else '❌'}")
    
    # 4. Estado general
    sistema_listo = archivos_ok and llm_disponible
    print(f"\n🎯 SISTEMA LISTO: {'✅ SÍ' if sistema_listo else '❌ NO'}")
    
    if sistema_listo:
        print("\n🚀 ¡Puedes ejecutar la comparación!")
        print("   Ejecuta: resultados = ejecutar_comparacion_completa()")
    else:
        print("\n⚠️ Configura los elementos faltantes antes de continuar")
    
    return sistema_listo

def mostrar_comandos_principales():
    """Muestra los comandos principales para usar"""
    print("""
🎯 COMANDOS PRINCIPALES:

🔧 CONFIGURACIÓN:
   verificar_estado_sistema()           # Verifica todo el sistema
   probar_conexion_llm()               # Prueba el LLM actual
   cambiar_proveedor_llm("openai")     # Cambia el proveedor

📊 EJECUCIÓN:
   resultados = ejecutar_comparacion_completa()      # Análisis completo
   resultados = comparacion_rapida_langchain()       # Análisis rápido
   resultados = analisis_por_dominio("legal")        # Análisis especializado

📈 UTILIDADES:
   stats = obtener_estadisticas_comparacion(resultados)  # Estadísticas
   reporte = generar_reporte_texto(resultados)           # Reporte texto
   exportar_resultados(resultados, "archivo.json")       # Exportar

🎨 VISUALIZACIÓN:
   mostrar_informacion_pdfs(resultados)      # Info de PDFs
   mostrar_resultados_similitud(resultados)  # Puntuaciones
   crear_grafico_comparacion(resultados)     # Gráfico de barras
   mostrar_palabras_importantes(resultados)  # Palabras clave TF-IDF
   mostrar_analisis_langchain(resultados)    # Análisis IA estructurado
""")

# Verificar estado del sistema al cargar
print("🚀 INICIALIZANDO SISTEMA...")
sistema_listo = verificar_estado_sistema()

# Mostrar comandos principales
mostrar_comandos_principales()

print("\n" + "=" * 60)
print("📚 COMPARADOR DE PDFs CON LANGCHAIN - LISTO PARA USAR")
print("=" * 60)

if sistema_listo:
    print("✅ Todo configurado correctamente")
    print("🎯 Siguiente paso: resultados = ejecutar_comparacion_completa()")
else:
    print("⚠️ Completa la configuración antes de usar")

print("✅ BLOQUE 17 - Sistema inicializado")

## 🎬 EJEMPLO DE EJECUCIÓN COMPLETA

In [ ]:
# Paso 1: Verificar sistema
print("🔍 Verificando sistema...")
if verificar_estado_sistema():
    
    # Paso 2: Ejecutar comparación completa
    print("🚀 Ejecutando comparación...")
    resultados = ejecutar_comparacion_completa()
    #resultados = comparacion_rapida_langchain()
    
    # Paso 3: Generar reporte
    if resultados and "error" not in resultados:
        print("📝 Generando reporte...")
        reporte = generar_reporte_texto(resultados)
        
        # Paso 4: Exportar resultados
        exportar_resultados(resultados, "comparacion_final.json")
        
        # Paso 5: Mostrar resumen
        stats = obtener_estadisticas_comparacion(resultados)
        print(f"🎯 Similitud final: {stats['resumen_general']['similitud_combinada']}")
        
        print("✅ ¡Comparación completada exitosamente!")
    else:
        print("❌ Error en la comparación")
else:
    print("⚠️ Configura el sistema antes de ejecutar")


print("🎬 Ejemplo de ejecución preparado (comentado)")
print("📝 Descomenta el código anterior para ejecutar automáticamente")

# FIN DEL CÓDIGO
print("\n🎉 ¡CÓDIGO COMPLETO CARGADO EXITOSAMENTE!")
print("📖 Total de bloques: 17")
print("🔧 Funciones creadas: 25+")
print("🎯 ¡Listo para comparar documentos PDF con IA!")